In [7]:
%load_ext autoreload
%aimport gempipe, gempipe.flowchart
%autoreload 1

In [16]:
from gempipe import Flowchart

file = open('flowcharts/part_1.flowchart', 'r')
header = 'flowchart TB \n'
flowchart = Flowchart(header + file.read())
file.close()
flowchart.render(height=300, zoom=2)

# Part 1: gempipe recon

`gempipe recon` is designed to reconstruct a **draft pan-GSMM** and a presence/absence matrix (**PAM**), starting either from genomes or proteomes. Below we show all the options: 

In [1]:
import subprocess

command = f"""gempipe recon -h"""
process = subprocess.Popen(command, shell=True)
response = process.wait()


usage: gempipe recon [-h] [-v] [-c] [-o] [--verbose] [--overwrite] [--dbs]
                     [-t] [-g] [-p] [-gb] [-s] [-b] [--buscoM] [--ncontigs]
                     [--N50] [--identity] [--coverage] [-rm] [-rp] [-rs] [-mc]
                     [--tcdb] [--dedup] [--norec]

gempipe v1.33.0, please cite "TODO". Full documentation available at
https://gempipe.readthedocs.io/en/latest/index.html.

optional arguments:
  -h, --help            Show this help message and exit.
  -v, --version         Show version number and exit.
  -c , --cores          Number of parallel processes to use. (default: 1)
  -o , --outdir         Main output directory (will be created if not
                        existing). (default: ./)
  --verbose             Make stdout messages more verbose, including debug
                        messages. (default: False)
  --overwrite           Delete the working/ directory at the startup.
                        (default: False)
  --dbs                 Path were t

## Starting from genomes

gempipe accepts both genomes and proteomes. Each file in input is assumed to belong to a different strain. There are several options to specifiy the input genomes: 

* Specify a **folder** containing the input genomes. They are all assumed to belong to the same species. For example: 

```bash
    gempipe recon -g my_genomes/
```

* Specify a list of input genome **files** (comma separated). They are all assumed to belong to the same species. For example: 

```bash
    gempipe recon -g my_genomes/GCA_001689725.1.fa,my_genomes/GCA_001756855.1.fa,my_genomes/GCA_003058285.2.fa
```

* Specify a list of input genome files, **grouped** by species. This follows a rigid sintax: `SpA@G1,G2,G3+SpB@G4,G5+SpC@G6,G7...`. For example:

```bash
    gempipe recon Eda@my_genomes/GCA_001689725.1.fa,my_genomes/GCA_001756855.1.fa,my_genomes/GCA_003058285.2.fa+Eap@my_genomes/GCA_016925695.1.fa,my_genomes/GCA_024169515.1.fa
```


## Starting from species taxids

As an alternative starting point, it's possible to insert the list of NCBI Species Taxids under study (comma separated). All the available genome assemblies under those taxids will be **automatically dowloaded** from NCBI, and grouped based on the species. For example: 

```bash
    gempipe recon -t 252393,68334
```

💡 **Tip!** Each species appearing on NCBI is linked to its specific NCBI Species Taxid. Suppose for example that you want to know the taxid of _Erwinia aphidicola_: enter the [NCBI Taxonomy Database](https://www.ncbi.nlm.nih.gov/taxonomy), and type the name of the species. Once you land on the webpage for the species, you can find the taxid right below its name: 68334 for _E. aphidicola_!

## Filtering the genomes

Bad-quality genomes in input are detected and **ignored** in subsequent analysis. Bad-quality is defined by 3 metrics, which can be either **techinical** or **biological**: 

1. The maximum **number of contigs** per genome (`--ncontigs`).
2. The minimum **N50** per genome (`--N50`).
3. The maximum number of Busco's **missing** single-copy orthologs per genome (`--buscoM`). It can be expressed also in percentage. 

Please note that the total number of evaluated Busco's single-copy orthologs depends on the selected Busco **database**. The default database is `bacteria_odb10`, but we strongly suggest to select a more **specific** database according to your organisms. This way, the number the evaluated Busco's single-copy orthologs will **increase**. To change the Busco database, use `-b`/`--buscodb`. 

💡 **Tip!** To read the list of available Busco databases, type `gempipe recon --buscodb show`.

For example, below we are dealing with some metagenome-derived assemblies, so we need to relax the default thresholds:

```bash
    gempipe recon -t 252393,68334 -b enterobacterales_odb10 --ncontigs 2000 --N50 5000 --buscoM 10%
```


## Starting from proteomes

gempipe gives its best when starting from genomes, as a gene recovery procedure is applied. Anyway, starting **directly** from proteomes is also allowed (see flowchart), if you already dispose of a reliable genome annotation. We define a proteome as a multifasta file containing all the strain-specific gene sequences written in aminoacidic format. There are several options to specifiy the input proteomes: 

* Specify a **folder** containing the input proteomes. They are all assumed to belong to the same species. For example:

```bash
    gempipe recon -p my_proteomes/
```

* Specify a list of input proteome **files** (comma separated). They are all assumed to belong to the same species. For example:

```bash
    gempipe recon -p my_proteomes/GCA_001689725.1.fa,my_proteomes/GCA_001756855.1.fa,my_proteomes/GCA_003058285.2.fa
```

* Specify a list of input proteome files, **grouped** by species. This follows a rigid sintax: `SpA@P1,P2,P3+SpB@P4,P5+SpC@P6,P7...`. For example:

```bash
    gempipe recon Eda@my_proteomes/GCA_001689725.1.fa,my_proteomes/GCA_001756855.1.fa,my_proteomes/GCA_003058285.2.fa+Eap@my_proteomes/GCA_016925695.1.fa,my_proteomes/GCA_024169515.1.fa
```

## Starting from genbanks

Proteomes can be also passed as genbank files. In this case, gempipe will try to recover gene annotations to boost the [Memote metrics](https://memote.readthedocs.io/en/latest/index.html) under the "gene annotation" section. There are several options to specifiy the input genbanks: 

* Specify a **folder** containing the input genbanks. They are all assumed to belong to the same species. For example:

```bash
    gempipe recon -gb my_genbanks/
```

* Specify a list of input genbank **files** (comma separated). They are all assumed to belong to the same species. For example:

```bash
    gempipe recon -gb my_genbanks/GCA_001689725.1.gb,my_genbanks/GCA_001756855.1.gb,my_genbanks/GCA_003058285.2.gb
```

* Specify a list of input genbank files, **grouped** by species. This follows a rigid sintax: `SpA@P1,P2,P3+SpB@P4,P5+SpC@P6,P7...`. For example:

```bash
    gempipe recon Eda@my_genbanks/GCA_001689725.1.gb,my_genbanks/GCA_001756855.1.gb,my_genbanks/GCA_003058285.2.gb+Eap@my_genbanks/GCA_016925695.1.gb,my_genbanks/GCA_024169515.1.gb
```

💡 **Tip!** Start from genbanks if you want to obtain the highest [Memote metrics](https://memote.readthedocs.io/en/latest/index.html), as gempipe will try to improve also the "gene annotation" section.

## Creation of a PAM

Proteins are **clustered** based on their amino acidic sequence **identity**. Each cluster has one **representative** sequence. 

Following the clustering, a presence/absence matrix (**PAM**) is derived. The PAM has clusters in row and strains in column. If a strain harbors two or more proteins belonging to the same cluster, those will appear in the same cell separated by a **semicolon** (`;`). As an example, we report below a PAM with just 3 strains.

In [9]:
import pandas as pnd

pnd.read_csv('tutoring_materials/aphidicola/pam.csv', index_col=0)#.tail()

,GCA_918698235.1,GCA_014773485.1,GCA_024169515.1
Cluster_0,LBGCNHPF_02539,OEIEKCCN_02232,OCILAMFM_00111
Cluster_1,NaN,OEIEKCCN_03649,OCILAMFM_04027
Cluster_2,NaN,OEIEKCCN_04206,NaN
Cluster_3,NaN,NaN,OCILAMFM_04171
Cluster_4,LBGCNHPF_03507,OEIEKCCN_03842,OCILAMFM_04138
...,...,...,...
Cluster_5475,LBGCNHPF_02700,OEIEKCCN_04289,OCILAMFM_01419
Cluster_5476,LBGCNHPF_01132,NaN,NaN
Cluster_5477,NaN,NaN,OCILAMFM_04217
Cluster_5478,LBGCNHPF_03644,OEIEKCCN_00956,NaN


## Gene recovery

When starting from genomes (`-t` or `-g`), gempipe executes 3 subsequent gene recovery modules. If you want to skip the gene recovery, activate the `--norec` option. To have more details on the gene recovery implementation, please read the Methods in the [gempipe paper](how_to_cite.ipynb).

* **Module 1.** Sometimes a gene in a genome undergo little mutations, such as base insertions, deletions or sobstitutions. This will probably lead to the formation of **premature stop** codons in the middle of the sequence. This in turn will lead to the prediction of two or more coding sequences, instead of one, from the same genomic region. Anyway, a similar behaviour can be also be due to sequencing or assembling **errors**. gempipe search for proteins broken in **two** high-identity pieces, and **assumes** the broke up to be due to technical (non-biological) issues. Sequences recovered in this way have a `_frag` suffix in the PAM. 

* **Module 2.** The prediction of coding sequences in a genome (aka gene calling) may not be perfect. Some genes may simply not be seen. To cope with this problem, gempipe searches for extra coding sequences by aligning representative sequences on the genome **masked** from its predicted genes. Like in all recovery modules, only high-identity and high-coverage alignments are considered. Sequences recovered in this way have a `_refound` suffix in the PAM. 

* **Module 3.** From a bioinformatics point of view, sometimes a coding sequence ends **internally** to another coding sequence. In these cases, gene callers may have problems in distinguishing the two overlapping genes. This Module briefly takes the short fragments recovered in Module 2, and seeks to extend them over the previously masked regions (where another gene was originally annotated by the gene caller). Sequences recovered in this way have a `_overlap` suffix in the PAM. 

⏩ **Warning!** gempipe gives its best when starting from genomes: starting from proteomes (multifasta or genbank) will **skip** all the gene recovery modules. 

## Draft pan-GSMM reference-free reconstruction

gempipe reconstructions are based on the [BiGG database](http://bigg.ucsd.edu). We prefer to work with the BiGG notation system because its IDs are **human readable** (campare for example the BiGG ID for glucose `glc__D` with `cpd00027`, its [ModelSEED](https://modelseed.org/biochem/compounds) counterpart).

gempipe uses the representative seuences of the clusters to create a reference-**free** draft pan-GSMM using the same [CarveMe](https://carveme.readthedocs.io/en/latest/index.html) gene database and **universes**, but applying different reconstruction rules. To choose between gram positive and gram negative reconstructions, use the `-s`/`--staining` option.

During the reconstruction process, only reactions with genetic support are included, and no automatic gap-filling is performed, this way **minimizing** the number of **false-positives**. This approach goes in the opposite direction compared to other famous tools like [CarveMe](https://carveme.readthedocs.io/en/latest/index.html), which include an _unskippable_ gap-filling algorithm needed to ensure "FBA-ready" models in output: this forced gap-filling might be the cause of the inclusion of false-positive reactions (please read the Results in the [gempipe paper](how_to_cite.ipynb)).

Note that each BiGG reaction can appear in several different BiGG models, and each of them may encode its GPR with a different **protein complex definition**. With `gempipe recon`, a reaction is copied from the selected universe if **at least one** of the alternative original protein complexes definitions is **fully** satisfied. Moreover, if two or more slightly different proteins align equally well on the same BiGG gene, all the involved GPRs will take into account these alternative **isoforms**. We believe that these two design decisions lead to more accurate GPRs, solving two known CarveMe issues, [#180](https://github.com/cdanielmachado/carveme/issues/180) and [#182](https://github.com/cdanielmachado/carveme/issues/182). 

Since the BiGG database contains **few** genes, and it's heavily **biased** towards model organisms, we try to recover extra alternative genes using a dedicated **functional annotation** with the state-of-the-art tool [eggNOG-mapper](http://eggnog-mapper.embl.de) (see flowchart). gempipe will take care of downloading all the needed databases.

## Draft pan-GSMM expanded reference-based reconstruction

gempipe accetps in input an **optional reference GSMM** (any type: pan or strain-specific): the reference GSMM itself is provided with `-rm`/`--refmodel`, while the associated proteome (aminoacidic multifasta) is provided with `-rp`/`--refproteome`. Providing a reference, the reference-free reconstruction previously produced (see above) is used to expand the reference itself with new genes and reactions. 

In "pure" reference-based reconstructions, **orthologs** are determined for each strain via a best-reciprocal hits alignment (**BRH**) with the reference, as described in the [2019 Nature Protocol Extension](https://doi.org/10.1038/s41596-019-0254-3) and implemented in a recent pipeline named [Bactabolize](https://doi.org/10.7554/eLife.87406.2). Then, a **copy** of the reference is **subtracted** from the genes missing of an ortholog, leading to the retention of reactions specific to the strain to model. Since it's **rare** to dispose **in advance** of a manually curated _pan-model_ for the species under study, usually the curated reference used in the BRH approach is a **strain-specific** model. This implies that the strain to model will harbour only a **subset** of the reference strain's reactions. This clearly limits the exploration of the biodiversity at the strain level. Therefore, reference-based reconstructions made with `gempipe recon` are always expanded with new reactions coming from the previously produced reference-free reconstruction (see flowchart).

First, for each strain to model, we perform a BRH with the reference. This enable us to "**translate**" the genes encoded in the reference GSMM to clusters ID (rows of the PAM). Then, the translated reference GSMM is **expanded** with new reactions coming from the reference-**free** reconstruction. After this expansion process, the expanded reference GSMM becomes a draft pan-GSMM (see flowchart). In these reference-based reconstructions, the reference is the cornerstone of the reconstruction, providing the biomass assembly reaction, the NGAM energy (non-growth associated maintainence energy), all the chemical formulas and charges assigned to metabolites, and so on. Therefore, the transfer of reactions from the reference-free reconstruction to the reference GSMM is not conducted blindly, but follows some principles: 

* If the reaction ID appears in the reference, we assume the reaction is already there. We just check if the GPR needs to be expanded with new genes.

* If the reaction ID doesn't appear in the reference, we search for a **synonym** reaction, defined as a reaction having the same reactant IDs and product IDs (ignoring protons). If a synonym is found in the reference, we assume the reaction is already there. Again, we just check if its GPR needs to be updated with new genes. 

* If it's impossible to find synonym in the reference, then the reaction is transferred. New metabolites and genes are also transferred if needed. 

During the reaction transfer, if a metabolite is encoded both in the reference and in the reference-free reconstruction with same ID but different formula and/or charge, we **maintain** the metabolite definition of the reference. To edit new reactions and metabolites coming from the reference-free reconstruction, it's possible to use the `-m`/`--mancor` option. This will provide a text file of **manual corrections** to be **superimposed** during the reference expansion, particularly useful to avoid unbalanced reactions and stoichiometric inconsistencies. It describes one rule per line, following a rigid syntax: `rule.ID:value`. Rules are divided in 4 categories:

* Formula of a metabolite, with the rule `formula`. For example:

```bash
    formula.isocapcoa:C27H42N7O17P3S
```

* Charge of a metabolties, with the rule `charge`. For example:

```bash
    charge.cdigmp:-2
```

* Definition of a reaction, with the rule `reaction`. For example:

```bash
    reaction.NTD12:dimp_c + h2o_c --> din_c + pi_c + h_c
```

* Exclusion of a reaction from the transfer, with the rule `blacklist`. For example:

```bash
    blacklist.LIPO3S24_BS
```

Plase note that metabolite IDs must be provided **without** compartment in the `formula` and `charge` rules. It is possible to comment a line (that is, to ignore a rule) prepending a `%` at the beginnning of the line. 

💡 **Tip!** Often, an artifact gene marking all the spontanous reactions is utilized in GSMMs, and we call it the "spontaneous" gene (often it is `spontaneous`, literally). It is useful to distinguish spontaneous reactions from reactions that remained without GPR. You can specify the "spontanous" gene adopted in the reference using `-rs`/`--refspont`.

## Transport reactions expansion with TCDB

Since the goodness of qualitative growth simulations usually depends on transport reactions, and since the BiGG gene database is small and biased towards model organisms, we provide an optional expansion of the tranport reactions using the `--tcdb` switch (please read Methods in the [gempipe paper](how_to_cite.ipynb) for the implementation details). 

Briefly, representative sequences of the clusters are aligned on the entire TCDB gene database. For many transport families we have generated in advance BiGG-compatible reaction strings that can be imported into a GSMM as new reactions, together with the dedicated GPR, only if all the components of the trasport protein complex have been detected. Be default, the addition of TCDB-derived transport reactions is disabled, as they could impare the stoichiometric consistency. 

## Annotation and duplicates removal

When a draft pan-GSMM has been obtained, it is subjected to metabolites and reactions **re-annnotation** (see flowchart). The annotation is performed de-novo using [MetaNetX](https://doi.org/10.1093/nar/gkaa992) v4.4 (MNX). This adds annotations for [HMDB](https://hmdb.ca), [KEGG](https://www.kegg.jp), [MetaCyc](https://metacyc.org), [Rhea](https://www.rhea-db.org), [ChEBI](https://www.ebi.ac.uk/chebi/), and many other databases. As a consequence, [Memote metrics](https://memote.readthedocs.io/en/latest/index.html) of the output GSMMs should improve.

Then, the MNX annotation is optionally used to detect and correct eventual duplicated metabolites and reactions in the draft pan-GSMM, activating the `--dedup` option. Duplicates are present right in the CarveMe universes, for example [`glc_D_B`](http://bigg.ucsd.edu/universal/metabolites/glc_D_B) and [`glc__bD`](http://bigg.ucsd.edu/universal/metabolites/glc__bD), and other could be formed after the reference expansion (please read the [gempipe paper](how_to_cite.ipynb) for further information). Duplicate metabolites are substituted with a replacement metabolite, selected giving **precedence** to metabolites contained in the reference GSMM, if provided. Please note that if the duplicate and the replacement metabolites are both present in the reference GSMM, they are both retained.

Next, if `--dedup` is active, the MNX annotation is used to detect **duplicate reactions**. Duplicates are filtered to have the same reactants and products IDs apart from protons. This prevents transport reactions for different compartments to be seen as duplicates. As for duplicate metabolites, duplicate reactions are substituted with a replacement reaction, selected giving **precedence** to those contained in the reference GSMM, if provided. If the duplicate and the replacement reactions are both present in the reference GSMM, they are both retained. Otherwise, the replacement reaction GPR **inherit** the genes coming from the duplicate. By default, the metabolites and reactions deduplication is disabled, as it could impare the stoichiometric consistency. 

The **re-annotated** draft-pan GSMM can be seen as the ultimate draft pan-GSMM, main output of `gempipe recon` together with the PAM.

## Output files

`gempipe recon` produces 3 **main** output files in the specified output directory (`-o`/`--outdir`). These are the minimum files required for manual curating the draft pan-GSMM using the [gempipe API for Python](autoapi/gempipe/curate/index) (see the function [gempipe.initialize](https://gempipe.readthedocs.io/en/latest/autoapi/gempipe/curate/gaps/index.html#gempipe.curate.gaps.initialize)):

* **draft_panmodel.json.** The draft pan-GSMM, which will likely need a manual curation (see [Part 2](part_2_manual_curation.ipynb)).

* **pam.csv.** The final presence/absence matrix (PAM), accounting for the gene recovery.

* **annotation.csv.** Functional annotation made by [eggNOG-mapper](http://eggnog-mapper.embl.de).

Moreover, gempipe produces also a working directory (`./working/`) with all the intermediete files and logs needed during reconstruction. The working directory allows gempipe to **restart** from the last completed task in case of interruption. To avoid the use of cached files (ie, perform a fresh run), utilize the `--overwrite` option. Please note that the working directory comprises also **~50 GB** of databases needed for the functional annotation. Some files in the working directory are higly informative, and are described below. 

* **filtering/tmetrics.csv.** Technical metrics (N50, n_contigs) used to filter good quality genomes.

* **filtering/bmetrics.csv.** Biological metrics (BUSCO) used to filter good quality genomes.

* **free/gpr_inflator/gid_to_cluster_all.txt.** Clusters grouped by functional annotation, used during the recovery of alternative genes.

* **expansion/results.csv.** Table listing the reactions of the reference-free reconstruction (`rid`) and their fate respect to the expanding reference (`action`). If a reaction ID or a synonym is found in the reference (`rid_found` or `synonym`, respectively), then the reaction will not be added (`action:ignore`), unless some new genes have to be handled to form a new GPR (`action:update_gpr`). If a reference reaction is found (either same ID or a synonym), then the presence of the exact same reactants and products (regardless protons) is checked (`same_mids`), as well as their formula and charge (`same_fc`). If the final reaction included ends up to be unbalaced, then balancing suggestions are included (`bal_suggestions`). 

* **duplicates/dup_m_edits.csv.** Table listing the duplicate metabolites (`duplicated`) and their replacing metabolite (`replacement`). Formula, charge, presence in the reference and presence in the reference-free reconstruction are reported both for the duplicate and the replacement. 

* **duplicates/dup_m_translations.csv.** Table listing the reactions (`rid`) affected by duplication removal, which are visible before (`reaction_old`) and after (`reaction_new`) the removal. Eventual new metabolites created appear under `adde_mids`. If the final reaction ends up to be unbalaced, then balancing suggestions are included (`bal_suggestions`). 

* **duplicates/dup_r_edits.csv.** Table listing the duplicate reactions (`duplicated`) and their replacing reaction (`replacement`). Reaction definition, GPR, presence in the reference and presence in the reference-free reconstruction are reported both for the duplicate and the replacement.